<a href="https://colab.research.google.com/github/iamstillalexh/Pico8AudioConverter/blob/main/Pico_8_PCM_Converter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1) Install FFmpeg

In [ ]:
#@title <font size="5">← ឵឵<i>Upgrade FFmpeg to v4.2.2</font> { vertical-output: true }
from IPython.display import clear_output
import os, urllib.request
HOME = os.path.expanduser("~")
pathDoneCMD = f'{HOME}/doneCMD.sh'
if not os.path.exists(f"{HOME}/.ipython/ttmg.py"):
    hCode = "https://raw.githubusercontent.com/yunooooo/gcct/master/res/ttmg.py"
    urllib.request.urlretrieve(hCode, f"{HOME}/.ipython/ttmg.py")

from ttmg import (
    loadingAn,
    textAn,
)

loadingAn(name="lds")
textAn("Installing Dependencies...", ty='twg')
os.system('pip install git+git://github.com/AWConant/jikanpy.git')
os.system('add-apt-repository -y ppa:jonathonf/ffmpeg-4')
os.system('apt-get update')
os.system('apt install mediainfo')
os.system('apt-get install ffmpeg')
clear_output()
print('Installation finished.')

Installation finished.


# 2) Install [p8scii-encoder](https://gitlab.com/dev_urandom/p8scii-encoder)

In [ ]:
!mkdir src

In [ ]:
#@title Create file from C

%%writefile src/main.c

#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <unistd.h>

#ifdef _WIN32
#include <windows.h>
#endif

const char* lowchars[16] = {"▮", "■", "□", "⁙", "⁘", "‖", "◀", "▶", "「", "」", "¥", "•", "、", "。", "゛", "゜" };

const char* highchars[128] = {
	"█", "▒", "🐱", "⬇️", "░", "✽", "●", "♥", "☉", "웃", "⌂", "⬅️", "😐", "♪", "🅾️", "◆", "…", "➡️", "★", "⧗", "⬆️", "ˇ", "∧", "❎", "▤", "▥", "あ", "い", "う", "え", "お", "か", "き", "く", "け", "こ", "さ", "し", "す", "せ", "そ", "た", "ち", "つ", "て", "と", "な", "に", "ぬ", "ね", "の", "は", "ひ", "ふ", "へ", "ほ", "ま", "み", "む", "め", "も", "や", "ゆ", "よ", "ら", "り", "る", "れ", "ろ", "わ", "を", "ん", "っ", "ゃ", "ゅ", "ょ", "ア", "イ", "ウ", "エ", "オ", "カ", "キ", "ク", "ケ", "コ", "サ", "シ", "ス", "セ", "ソ", "タ", "チ", "ツ", "テ", "ト", "ナ", "ニ", "ヌ", "ネ", "ノ", "ハ", "ヒ", "フ", "ヘ", "ホ", "マ", "ミ", "ム", "メ", "モ", "ヤ", "ユ", "ヨ", "ラ", "リ", "ル", "レ", "ロ", "ワ", "ヲ", "ン", "ッ", "ャ", "ュ", "ョ", "◜", "◝"};

int main(int argc, char** argv) {

	int c;

	FILE* infile = stdin;
	FILE* outfile = stdout;

	const char* inname = NULL;
	const char* outname = NULL;

#ifdef _WIN32
	HWND console = GetConsoleWindow();

	if (!console) {
		if (argc > 1) {
		} else {

			// use GetOpenFileName to select the file
			OPENFILENAME ofn;
			TCHAR szFile[PATH_MAX] = {0};

			memset(&ofn,0,sizeof ofn);
			ofn.lStructSize = sizeof ofn;
			ofn.lpstrFile = szFile;
			ofn.nMaxFile = PATH_MAX;
			ofn.lpstrTitle = "Which file should be converted into a PICO-8 string?";

			if (GetOpenFileName(&ofn) == TRUE) {
				inname = strdup(ofn.lpstrFile);
			} else {

				MessageBox(NULL,"Please select a file to be converted using the dialog box or drag its icon onto this program's executable.","P8SCII Encoder",MB_OK | MB_ICONERROR);
				return 1;

			}

		}
	}
#endif

	int opt;
	while ((opt = getopt(argc,argv,"ho:")) != -1) {
		switch(opt) {
			case 'o':
				outname = optarg;
				break;
			case 'h':
			case '?':
				fprintf(stderr,"Usage: %s [-o output_file] [input_file]\n",argv[0]);
				return 1;
				break;
		}
	}

	if (optind < argc) {
		inname = argv[optind];
	}

	if (inname) {
		infile = fopen(inname,"rb");
		if (!infile) { perror("Unable to open input file"); return 1;}

		if (!outname) {
			char* newout = malloc(strlen(inname) + 4 + 1);
			if (!newout) { puts("Unable to allocate memory for output filename!"); return 1; }
			strcpy(newout,inname);
			strcat(newout,".txt");
			outname = newout;
		}
	}

	if (outname) {
		outfile = fopen(outname,"wb");
		if (!outfile) { perror("Unable to open input file"); return 1;}
	}

	fputs("\"",outfile);
	while ((c = fgetc(infile)) != EOF) {
		if ((c >= 16) && (c < 32)) 
			fputs(lowchars[c-16],outfile);
		else if (c >= 128) 
			fputs(highchars[c-128],outfile);
		else switch(c) {
			case 0: fputs("\\000",outfile); break;
			case 1: fputs("¹",outfile); break;
			case 2: fputs("²",outfile); break;
			case 3: fputs("³",outfile); break;
			case 4: fputs("⁴",outfile); break;
			case 5: fputs("⁵",outfile); break;
			case 6: fputs("⁶",outfile); break;
			case 7: fputs("⁷",outfile); break;
			case 8: fputs("⁸",outfile); break;
			case 9: fputs("\\t",outfile); break;
			case 10: fputs("\\n",outfile); break;
			case 11: fputs("ᵇ",outfile); break;
			case 12: fputs("ᶜ",outfile); break;
			case 13: fputs("\\r",outfile); break;
			case 14: fputs("ᵉ",outfile); break;
			case 15: fputs("ᶠ",outfile); break;
			case 92: fputs("\\\\",outfile); break;
			case 34: fputs("\\\"",outfile); break;
			default: fputc(c, outfile); break;
		}
	}
	fputs("\"",outfile);

	if (infile != stdin) fclose(infile);
	if (outfile != stdout) fclose(outfile);

	return 0;
}


Writing src/main.c


In [ ]:
#@title Create makefile

%%writefile Makefile

-include Makefile.cfg

OBJDIR=obj
SRCDIR=src

_CC=gcc
LD=ld
CFLAGS = -std=c11 -Wall
DEFS += -D_POSIX_C_SOURCE=2
LDFLAGS =

_OBJS = main.o

ifdef WINDOWS
CCPREFIX=x86_64-w64-mingw32-
OBJSUFFIX=.exe
LDFLAGS=-mwindows
endif

ifdef PROFILE
CFLAGS += -pg
LDFLAGS += -pg
endif

ifdef RELEASE
DEFS += -xSSE3 -O3 -DNDEBUG
else
DEFS += -g -O0
endif

CC = $(CCPREFIX)$(_CC)
OBJS = $(patsubst %,$(OBJDIR)/%,$(_OBJS))

install: p8scii-encoder$(OBJSUFFIX)

p8scii-encoder$(OBJSUFFIX): $(OBJS)
	$(CC) -o $@ $^ $(DEFS) $(LDFLAGS)

$(OBJDIR)/%.o: $(SRCDIR)/%.c $(OBJDIR)
	$(CC) -c -o $@ $< $(CFLAGS) $(DEFS)

$(OBJDIR):
	mkdir -p $(OBJDIR)

clean:
	rm -rf $(OBJDIR)/*.o p8scii-encoder

.PHONY: clean install

Writing Makefile


In [ ]:
%%shell

make

mkdir -p obj
gcc -c -o obj/main.o src/main.c -std=c11 -Wall -D_POSIX_C_SOURCE=2 -g -O0
gcc -o p8scii-encoder obj/main.o -D_POSIX_C_SOURCE=2 -g -O0 


# 3) Upload file and convert
Output should appear as both .raw and .txt files, the .txt is what you want

In [ ]:
import os, sys, re
input_filepath = "burymewithmymoney.mp3" #@param {type:"string"}
output_name = "buryme_pico" #@param {type:"string"}
!ffmpeg -i "$input_filepath" -f u8 -c:a pcm_u8 -ar 5512 -ac 1 "$output_name".raw
!./p8scii-encoder "$output_name".raw

ffmpeg version 4.3.2-0york0~18.04 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version='0york0~18.04' --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-avresample --disable-filter=resample --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librsvg --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab